# 第 8 章：语言模型

> 用 RNN 预测下一个词，生成文本

---

## 本章目标

- [ ] 理解语言模型的基本概念
- [ ] 用 RNN/LSTM 实现字符级语言模型
- [ ] 理解困惑度 (Perplexity) 评估指标
- [ ] 实现文本生成（采样策略）

---

## 什么是语言模型？

语言模型的目标是学习**序列的概率分布**：

$$P(w_1, w_2, ..., w_T) = \prod_{t=1}^{T} P(w_t | w_1, ..., w_{t-1})$$

简单说：预测下一个词/字符的概率。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

---

## 第一部分：数据准备

### 1.1 字符级语言模型

In [ ]:
class CharDataset:
    """
    字符级数据集
    """
    
    def __init__(self, text, seq_len=50):
        self.text = text
        self.seq_len = seq_len
        
        # 构建字符词表
        self.chars = sorted(list(set(text)))
        self.char_to_idx = {ch: i for i, ch in enumerate(self.chars)}
        self.idx_to_char = {i: ch for i, ch in enumerate(self.chars)}
        self.vocab_size = len(self.chars)
        
        # 将文本转为索引
        self.data = [self.char_to_idx[ch] for ch in text]
    
    def __len__(self):
        return len(self.data) - self.seq_len
    
    def __getitem__(self, idx):
        x = self.data[idx:idx + self.seq_len]
        y = self.data[idx + 1:idx + self.seq_len + 1]
        return torch.tensor(x), torch.tensor(y)


# 示例文本
sample_text = """
To be, or not to be, that is the question:
Whether 'tis nobler in the mind to suffer
The slings and arrows of outrageous fortune,
Or to take arms against a sea of troubles,
And by opposing end them. To die: to sleep;
No more; and by a sleep to say we end
The heart-ache and the thousand natural shocks
That flesh is heir to, 'tis a consummation
Devoutly to be wish'd. To die, to sleep;
To sleep: perchance to dream: ay, there's the rub;
"""

dataset = CharDataset(sample_text, seq_len=30)

print(f"文本长度: {len(sample_text)}")
print(f"词表大小: {dataset.vocab_size}")
print(f"字符: {dataset.chars[:20]}...")

# 查看一个样本
x, y = dataset[0]
print(f"\n输入: {''.join([dataset.idx_to_char[i.item()] for i in x])}")
print(f"目标: {''.join([dataset.idx_to_char[i.item()] for i in y])}")

---

## 第二部分：模型实现

### 2.1 字符级 LSTM 语言模型

In [ ]:
class CharLSTM(nn.Module):
    """
    字符级 LSTM 语言模型
    
    输入: 字符索引序列
    输出: 下一个字符的概率分布
    """
    
    def __init__(self, vocab_size, embed_dim, hidden_size, num_layers=2, dropout=0.2):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # 字符嵌入
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # LSTM
        self.lstm = nn.LSTM(
            embed_dim, hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # 输出层
        self.fc = nn.Linear(hidden_size, vocab_size)
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, hidden=None):
        """
        前向传播
        
        x: (batch, seq_len) 字符索引
        """
        # 嵌入
        embedded = self.dropout(self.embedding(x))  # (batch, seq, embed)
        
        # LSTM
        output, hidden = self.lstm(embedded, hidden)  # (batch, seq, hidden)
        
        # 输出层
        output = self.dropout(output)
        logits = self.fc(output)  # (batch, seq, vocab_size)
        
        return logits, hidden
    
    def init_hidden(self, batch_size, device):
        """初始化隐状态"""
        h = torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)
        c = torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)
        return (h, c)


# 创建模型
model = CharLSTM(
    vocab_size=dataset.vocab_size,
    embed_dim=64,
    hidden_size=128,
    num_layers=2,
    dropout=0.2
).to(device)

print(model)
print(f"\n总参数量: {sum(p.numel() for p in model.parameters()):,}")

### 2.2 训练

In [ ]:
def train_language_model(model, dataset, n_epochs=50, batch_size=32, lr=0.002):
    """
    训练语言模型
    """
    from torch.utils.data import DataLoader
    
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    losses = []
    
    for epoch in range(n_epochs):
        model.train()
        total_loss = 0
        
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            
            # 前向传播
            logits, _ = model(x)
            
            # 计算损失
            # logits: (batch, seq, vocab) → (batch*seq, vocab)
            # y: (batch, seq) → (batch*seq,)
            loss = criterion(logits.view(-1, dataset.vocab_size), y.view(-1))
            
            # 反向传播
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(dataloader)
        losses.append(avg_loss)
        
        if (epoch + 1) % 10 == 0:
            perplexity = np.exp(avg_loss)
            print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}, Perplexity = {perplexity:.2f}")
    
    return losses


# 训练
print("开始训练...")
losses = train_language_model(model, dataset, n_epochs=50)

# 可视化
plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('语言模型训练损失')
plt.grid(True, alpha=0.3)
plt.show()

---

## 第三部分：文本生成

### 3.1 采样策略

In [ ]:
def generate_text(model, dataset, start_text, length=100, temperature=1.0):
    """
    生成文本
    
    参数:
        start_text: 起始文本
        length: 生成长度
        temperature: 温度参数 (越低越确定，越高越随机)
    """
    model.eval()
    
    # 将起始文本转为索引
    chars = [dataset.char_to_idx.get(ch, 0) for ch in start_text]
    x = torch.tensor([chars]).to(device)
    
    generated = list(start_text)
    hidden = model.init_hidden(1, device)
    
    with torch.no_grad():
        # 先处理起始文本，更新隐状态
        logits, hidden = model(x, hidden)
        
        # 逐字符生成
        for _ in range(length):
            # 获取最后一个时间步的输出
            probs = F.softmax(logits[0, -1] / temperature, dim=0)
            
            # 采样
            next_idx = torch.multinomial(probs, 1).item()
            next_char = dataset.idx_to_char[next_idx]
            generated.append(next_char)
            
            # 准备下一个输入
            x = torch.tensor([[next_idx]]).to(device)
            logits, hidden = model(x, hidden)
    
    return ''.join(generated)


# 测试不同温度
print("=" * 60)
print("文本生成（不同温度）")
print("=" * 60)

for temp in [0.5, 1.0, 1.5]:
    print(f"\nTemperature = {temp}:")
    print("-" * 40)
    text = generate_text(model, dataset, "To be", length=100, temperature=temp)
    print(text)

---

## 第四部分：困惑度 (Perplexity)

### 4.1 困惑度的定义

困惑度是语言模型的标准评估指标：

$$\text{Perplexity} = \exp\left(-\frac{1}{N}\sum_{i=1}^{N}\log P(w_i|w_{<i})\right)$$

直觉：模型对每个词的"平均惊讶程度"
- 困惑度 = 1：完美预测
- 困惑度 = V（词表大小）：随机猜测

In [ ]:
def compute_perplexity(model, dataset):
    """
    计算困惑度
    """
    model.eval()
    criterion = nn.CrossEntropyLoss()
    
    total_loss = 0
    total_count = 0
    
    with torch.no_grad():
        for i in range(0, len(dataset), 32):
            batch = [dataset[j] for j in range(i, min(i+32, len(dataset)))]
            x = torch.stack([b[0] for b in batch]).to(device)
            y = torch.stack([b[1] for b in batch]).to(device)
            
            logits, _ = model(x)
            loss = criterion(logits.view(-1, dataset.vocab_size), y.view(-1))
            
            total_loss += loss.item() * x.size(0) * x.size(1)
            total_count += x.size(0) * x.size(1)
    
    avg_loss = total_loss / total_count
    perplexity = np.exp(avg_loss)
    
    return perplexity


ppl = compute_perplexity(model, dataset)
print(f"困惑度: {ppl:.2f}")
print(f"解释: 模型在预测每个字符时，平均有 {ppl:.1f} 个可能的选择")

---

## 本章总结

1. **语言模型**：预测下一个词/字符的概率分布
2. **训练目标**：最大化似然（最小化交叉熵）
3. **评估指标**：困惑度 $\text{PPL} = \exp(\text{Loss})$
4. **生成策略**：温度采样控制随机性

---

## 下一章预告

**第 9 章：Seq2Seq 架构**

👉 [09_seq2seq_architecture.ipynb](./09_seq2seq_architecture.ipynb)